### 1. General info of dataset GSE235063

This is the Jupyter Notebook for dataset GSE235063. Its dataset includes barcodes/genes/matrix files for each sample.

Thus, we need to simply incorparate these barcodes/genes/matrix files and generate an AnnData object for each sample. 

In total, there are 75 acute myeloid leukemia (AML) samples. The datasets includes both raw and processed information (thus, a total of 150 samples).

<span style="color:green">**[DX]**</span> samples from diagnosis

<span style="color:green">**[REM]**</span> samples from remission

<span style="color:green">**[REL]**</span> samples from relapse 